<a href="https://colab.research.google.com/github/einsteinxx/UCLA_BIOENG_596_FALL_21/blob/main/zip_load_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import gc  #debug memory leaks in matplotlib
import csv #read in description files
import random #used to select random slice for patches
import cv2
import re #regexp for BUSI files
from skimage import exposure  #image equalization

!pip3 install -q torchinfo
!pip3 install -Uqq ipdb
################################################################################
#ULTRASOUND NEEDS
import PIL
#from PIL import Image
# Open the image form working directory
#image = Image.open(full_file)
from matplotlib import image
from ast import literal_eval #used to break out bounding boxes from strings
import time
import torch
import torchvision
import torchinfo

from torchinfo import summary
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

from numpy import clip

import torchinfo


#import ipdb  #debugger tool 
################################################################################

eps = 1e-16 #keep div by 0 away


from google.colab.patches import cv2_imshow
#
# Read Data from google drive
#
from google.colab import drive #for loading gdrive data
from google.colab import files

# install dependencies not included by Colab
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip3 install -q pydicom 
!pip3 install -q tqdm 
!pip3 install -q imgaug
!pip3 install -q pickle5

import pydicom #to read dicom files
from pydicom import dcmread
import pickle5 as pickle; #generic storage of image array

################################################################################
# Load data from google drive
################################################################################
drive.mount('/content/gdrive')

#
# Setup of input and output drives
#

#
# Choose training sets to use
#
training_set = 0 #0=UCLA US, 1 = UCLA + BUSI, 2 = BUSI

#storage areas 
data_dir = '/content/gdrive/Shareddrives/BreastUS'
local_dir = '/content/gdrive/My Drive/BreastUS' #for local storage
model_dir = '/content/gdrive/My Drive/BreastUS/MODEL_SAVE'
tensorboard_dir =  '/content/gdrive/My Drive/BreastUS/TENSORBOARD_SUMMARIES'


#
# UCLA specific data folders
#
csv_dir = os.path.join(data_dir,'Annotated data')

annotated_dir = os.path.join(data_dir,'Annotated data')

data_files = os.listdir(annotated_dir)
label_data_dir = os.path.join(annotated_dir,'LabelMe_3.0_format_updated')
label_files = os.listdir(label_data_dir)

#
# Alternative set of Ultrasound data
#
use_USnorm = 1
if (use_USnorm == 1): #use norm based on US data, not imagenet

    #busi_main_dir = '/content/gdrive/My Drive/BreastUS/BUSI_DATA' #downloaded BUSI
    busi_main_dir = '/content/gdrive/My Drive/BreastUS/0TO1NORM/BUSI_DATA_CONTRAST_EQUALIZED'
    #output converted BUSI images go in here:
    ucla_converted_main_dir = '/content/gdrive/My Drive/BreastUS/0TO1NORM/UCLA_DATA_CONTRAST_EQUALIZED'
else:
    #busi_main_dir = '/content/gdrive/My Drive/BreastUS/BUSI_DATA' #downloaded BUSI
    busi_main_dir = '/content/gdrive/My Drive/BreastUS/N2TO2_NORM/BUSI_DATA_CONTRAST_EQUALIZED'
    #output converted BUSI images go in here:
    ucla_converted_main_dir = '/content/gdrive/My Drive/BreastUS/N2TO2_NORM/UCLA_DATA_CONTRAST_EQUALIZED'    

#
# Stored output images
#
ucla_output_image_dir = '/content/gdrive/My Drive/BreastUS/UCLA_OUTPUT_IMAGES'


create_zip = 0 #make the zip file
load_zip = 1 #unzip to the VM area

################################################################################
#      GPU OPTIONS
### Enable GPU, if present
################################################################################
train_on_gpu = torch.cuda.is_available()
if (train_on_gpu):
    !nvidia-smi -L
    !nvidia-smi 
    dev=torch.device("cuda")
else:
    print('GPU NOT FOUND!!! USING CPU INSTEAD!!!!!')
    dev=torch.device("cpu")

     |████████████████████████████████| 793 kB 11.9 MB/s 
     |████████████████████████████████| 381 kB 56.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.
     |████████████████████████████████| 2.0 MB 13.5 MB/s 
     |████████████████████████████████| 256 kB 12.6 MB/s 
Mounted at /content/gdrive
GPU NOT FOUND!!! USING CPU INSTEAD!!!!!


#ZIP DIRECTORY STRUCTURE


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Usage: 7z <command> [<switches>...] <archive_name> [<file_names>...]
       [<@listfiles...>]

<Commands>
  a : Add files to archive
  b : Benchmark
  d : Delete files from archive
  e : Extract files from archive (without using directory names)
  h : Calculate hash values for files
  i : Show information about supported formats
  l : List contents of archive
  rn : Rename files in archive
  t : Test integrity of archive
  u : Update files to archive
  x : eXtract files with full paths

<Switches>
  -- : Stop switches parsing
  -ai[r[-|0]]{@listfile|!wildcard} : Include archives
  -ax[r[-|0]]{@listfile|!wildcard} : eXclude archives
  -ao{a|s|t|u} : set Overwrite mode
  -an : disable archive_name field
  -bb[0-3] : set output log level
  -bd : disable progress indicator
  -bs{o|e|p}{0|1|2} : 

In [ ]:
if (create_zip == 1):
    !7z a '/content/gdrive/My Drive/BreastUS/ucla_0TO1_data1.zip' '/content/gdrive/My Drive/BreastUS/0TO1NORM/UCLA_DATA_CONTRAST_EQUALIZED'

if (load_zip == 1):
    if (os.path.exists('/content/sample_data/UCLA_DATA')):
        pass
    else:
        os.mkdir('/content/sample_data/UCLA_DATA')
    !7z x -y '/content/gdrive/My Drive/BreastUS/ucla_0TO1_data1.zip' -o'/content/sample_data/UCLA_DATA'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs AMD EPYC 7B12 (830F10),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/My Drive/BreastUS/                                            1 file, 109833574245 bytes (103 GiB)

Extracting archive: /content/gdrive/My Drive/BreastUS/ucla_0TO1_data1.zip
 21% 4096 Open               87% 16384 Open               --
Path = /content/gdrive/My Drive/BreastUS/ucla_0TO1_data1.zip
Type = zip
Physical Size = 109833574245
64-bit = +

  0%      0% 4 - UCLA_DATA_CONTRAST_EQUALIZED/1_07 . 01.PNG_600_800_105_569_134_721.pck                                                                               

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls 'UCLA_DATA/UCLA_DATA_CONTRAST_EQUALIZED/1_076up782_a_dcc4675o_0.mp4-2021_09_03_16_33_59-labelme 3.0.zip'


default
